In [ ]:
!pip3 install beautifulsoup4
!pip3 install requests

import sys
import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

In [ ]:
def date_time(table_cells):
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out
                  
def landing_status(table_cells):
    out = [i for i in table_cells.strings][0]
    return out
                  
def get_mass(table_cells):
    mass = unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass = mass[0:mass.find("kg")+2]
    else:
        new_mass = 0
        return new_mass
                  
def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
   # colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    #if not(colunm_name.strip().isdigit()):
     #   colunm_name = colunm_name.strip()
      #  return colunm_name    

    try:
        colunm_name = ' '.join(row.contents)
        # Filter the digit and empty names
        if not(colunm_name.strip().isdigit()):
            colunm_name = colunm_name.strip()
            return colunm_name 
    except:
        pass

In [ ]:
url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"
response = requests.get(url)

soup = BeautifulSoup(response.text, 'html.parser')

print(soup.prettify())

In [ ]:
tables = soup.find_all('table')
first_launch_table = tables[2]

print(first_launch_table)

In [ ]:
column_names = []

colnames = soup.find_all('th')
for x in range (len(colnames)):
    name2 = extract_column_from_header(colnames[x])
    if (name2 is not None and len(name2) > 3):  
        column_names.append(name2) 

print(column_names)

In [ ]:
launch_dict= dict.fromkeys(column_names)

del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

In [ ]:
extracted_row = 0
#Extract each table 
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # get table row 
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number 
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        #get table element 
        row=rows.find_all('td')
        #if it is number save cells in a dictonary 
        if flag:
            extracted_row += 1
            # Flight Number value
            # TODO: Append the flight_number into launch_dict with key `Flight No.`
            launch_dict["Flight No."].append(flight_number)
            print(flight_number)
            datatimelist=date_time(row[0])
            
            # Date value
            # TODO: Append the date into launch_dict with key `Date`
            date = datatimelist[0].strip(',')
            launch_dict["Date"].append(date)
            #print(date)
            
            # Time value
            # TODO: Append the time into launch_dict with key `Time`
            time = datatimelist[1]
            launch_dict["Time"].append(time)
            #print(time)
              
            # Booster version
            # TODO: Append the bv into launch_dict with key `Version Booster`
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
            launch_dict["Version Booster"].append(bv)
            #print(bv)
            
            # Launch Site
            # TODO: Append the bv into launch_dict with key `Launch Site`
            launch_site = row[2].a.string
            launch_dict["Launch site"].append(launch_site)
            #print(launch_site)
            
            # Payload
            # TODO: Append the payload into launch_dict with key `Payload`
            payload = row[3].a.string
            launch_dict["Payload"].append(payload)
            #print(payload)
            
            # Payload Mass
            # TODO: Append the payload_mass into launch_dict with key `Payload mass`
            payload_mass = get_mass(row[4])
            launch_dict["Payload mass"].append(payload_mass)
            #print(payload_mass)
            
            # Orbit
            # TODO: Append the orbit into launch_dict with key `Orbit`
            orbit = row[5].a.string
            launch_dict["Orbit"].append(orbit)
            #print(orbit)
            
            # Customer
            # TODO: Append the customer into launch_dict with key `Customer`
            customer = list(row[6].strings)[0]
            launch_dict["Customer"].append(customer)
            print(customer)
            
            #if(customer is not None and len(customer) >= 1):
                #launch_dict["Customer"].append(customer)
                #print(customer)
            #else:
                #print ("Empty Cust")
                #launch_dict["Customer"] = ["blank"]
            
                
            # Launch outcome
            # TODO: Append the launch_outcome into launch_dict with key `Launch outcome`
            launch_outcome = list(row[7].strings)[0]
            launch_dict["Launch outcome"].append(launch_outcome)
            #print(launch_outcome)
            
            # Booster landing
            # TODO: Append the launch_outcome into launch_dict with key `Booster landing`
            booster_landing = landing_status(row[8])
            launch_dict["Booster landing"].append(booster_landing)
            #print(booster_landing)
                

In [ ]:
count1 = len(launch_dict['Booster landing'])
print(count1)

count2 =len(launch_dict['Flight No.'])
print(count2)

count3 =len(launch_dict['Launch site'])
print(count3)

count4 =len(launch_dict['Payload'])
print(count4)

count5 =len(launch_dict['Payload mass'])
print(count5)

count6 =len(launch_dict['Orbit'])
print(count6)

count7 =len(launch_dict['Customer'])
print(count7)

count8 =len(launch_dict['Launch outcome'])
print(count8)

count9 =len(launch_dict['Version Booster'])
print(count9)

count10 =len(launch_dict['Date'])
print(count10)

count11 =len(launch_dict['Time'])
print(count11)
df=pd.DataFrame(launch_dict)

df.head()

In [ ]:
df.to_csv('spacex_web_scrapped.csv', index=False)